<a href="https://colab.research.google.com/github/Luisasdasdsad/EjercicioTriangulo/blob/main/Copia_de_H2Olmos_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DATASET

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


def crear_datos_empresariales_completos():
    np.random.seed(42)
    n_meses = 48
    fechas = pd.date_range(start='2022-01-01', periods=n_meses, freq='M')
    t = np.arange(n_meses)

    # Variables operativas principales
    sedimentacion = 32 + 0.38 * t + 2 * np.sin(2 * np.pi * t / 12) + np.random.normal(0, 1.2, n_meses)
    sedimentacion = np.clip(sedimentacion, 30, 45)

    precipitacion = 120 + 25 * np.sin(2 * np.pi * t / 6) + np.random.normal(0, 20, n_meses)
    precipitacion = np.clip(precipitacion, 70, 170)

    capacidad_actual = 68 - 0.32 * t - 0.8 * (sedimentacion - 32) + np.random.normal(0, 1.5, n_meses)
    capacidad_actual = np.clip(capacidad_actual, 50, 75)

    # Variables financieras y de riesgo
    dscr = 1.22 - 0.0032 * t + 0.012 * (capacidad_actual - 60) + np.random.normal(0, 0.02, n_meses)
    dscr = np.clip(dscr, 1.05, 1.30)

    ebitda = 89 + 0.1 * np.sin(2 * np.pi * t / 6) + np.random.normal(0, 0.6, n_meses)
    ebitda = np.clip(ebitda, 85, 95)

    morosidad = 3.0 + 0.15 * t - 0.1 * (capacidad_actual - 60) + np.random.normal(0, 0.5, n_meses)
    morosidad = np.clip(morosidad, 2, 10)

    volumen_entregado = 88 - 0.23 * t + 0.1 * np.sin(2 * np.pi * t / 12) + np.random.normal(0, 1.1, n_meses)
    volumen_entregado = np.clip(volumen_entregado, 75, 95)

    # Capacidad futura (objetivo de predicción)
    capacidad_futura = capacidad_actual - 0.25 - 0.05 * (sedimentacion - np.mean(sedimentacion))
    capacidad_futura += np.random.normal(0, 1.0, n_meses)
    capacidad_futura = np.clip(capacidad_futura, 48, 72)

    # Clientes en riesgo (variable categórica)
    clientes_riesgo = np.ones(n_meses)
    clientes_riesgo[12:24] = 2
    clientes_riesgo[24:36] = 3
    clientes_riesgo[36:] = 4

    df = pd.DataFrame({
        'fecha': fechas,
        'mes': [f.month for f in fechas],
        'año': [f.year for f in fechas],
        'trimestre': [f.quarter for f in fechas],
        'sedimentacion_porcentaje': sedimentacion,
        'precipitacion_mm': precipitacion,
        'capacidad_util_porcentaje': capacidad_actual,
        'capacidad_futura_3m': np.roll(capacidad_futura, -3),
        'dscr': dscr,
        'ebitda_millones_soles': ebitda,
        'morosidad_porcentaje': morosidad,
        'volumen_agua_entregado_millones_m3': volumen_entregado,
        'clientes_riesgo_alto': clientes_riesgo.astype(int)
    })

    return df.iloc[:-3]

df_clean = crear_datos_empresariales_completos()
# print("DATOS EMPRESARIALES CARGADOS")
# print(f"   • Período: {df_clean['fecha'].iloc[0].strftime('%b %Y')} - {df_clean['fecha'].iloc[-1].strftime('%b %Y')}")
# print(f"   • Muestras: {len(df_clean)} meses")
# print(f"   • Capacidad: {df_clean['capacidad_util_porcentaje'].iloc[0]:.0f}% → {df_clean['capacidad_util_porcentaje'].iloc[-1]:.0f}%")

# ENTRENAMIENTO DEL MODELO PREDICTIVO

In [5]:
# Preparar datos para ML
features = [
    'sedimentacion_porcentaje', 'precipitacion_mm', 'volumen_agua_entregado_millones_m3',
    'dscr', 'ebitda_millones_soles', 'morosidad_porcentaje', 'clientes_riesgo_alto', 'mes'
]
target = 'capacidad_futura_3m'

X = df_clean[features]
y = df_clean[target]

# División temporal
split_idx = int(len(X) * 0.75)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# print(f"CONFIGURACIÓN DEL MODELO:")
# print(f"   • Variables: {len(features)} features")
# print(f"   • Entrenamiento: {len(X_train)} meses")
# print(f"   • Prueba: {len(X_test)} meses")

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelo
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled, y_train)

# Predicciones y evaluación
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"\nRESULTADOS DEL MODELO:")
print(f"   • R² Entrenamiento: {r2_train:.3f}")
print(f"   • R² Prueba: {r2_test:.3f}")
print(f"   • Error Absoluto Medio: {mae_test:.3f}%")


RESULTADOS DEL MODELO:
   • R² Entrenamiento: 0.985
   • R² Prueba: -0.037
   • Error Absoluto Medio: 1.075%


# DASHBOARD INTERACTIVO

In [6]:
def crear_dashboard_interactivo_completo(df_clean, model, scaler, features):
    # Estilos CSS personalizados
    estilo_completo = """
    <style>
    .titulo-dashboard {
        background: linear-gradient(90deg, #1e3c72, #2a5298);
        color: white;
        padding: 25px;
        border-radius: 15px;
        text-align: center;
        margin-bottom: 25px;
        box-shadow: 0 4px 12px rgba(0,0,0,0.15);
    }
    .card {
        background: white;
        padding: 20px;
        border-radius: 12px;
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
        margin: 15px 0;
        border-left: 6px solid #2a5298;
    }
    .kpi-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 20px;
        border-radius: 12px;
        text-align: center;
        box-shadow: 0 4px 12px rgba(0,0,0,0.15);
    }
    .alerta-roja {
        background: linear-gradient(135deg, #ff6b6b, #ee5a52);
        color: white;
    }
    .alerta-amarilla {
        background: linear-gradient(135deg, #ffd93d, #ffcd38);
        color: #333;
    }
    .alerta-verde {
        background: linear-gradient(135deg, #6bcf7f, #4caf50);
        color: white;
    }
    .simulador-section {
        background: #f8f9fa;
        padding: 25px;
        border-radius: 12px;
        border: 2px solid #e9ecef;
    }
    .control-panel {
        background: white;
        padding: 20px;
        border-radius: 10px;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    }
    </style>
    """

    display(HTML(estilo_completo))

    # Título principal del dashboard
    display(HTML("""
    <div class="titulo-dashboard">
        <h1 style="margin: 0; font-size: 32px;">SISTEMA DE INTELIGENCIA PREDICTIVA</h1>
        <h2 style="margin: 10px 0; font-size: 24px;">H2OImos S.A. - Gestión Estratégica del Embalse</h2>
        <p style="margin: 0; opacity: 0.9;">Dashboard Ejecutivo de Monitoreo y Alertas Tempranas</p>
    </div>
    """))

    # =========================================================================
    # SECCIÓN 1: KPI PRINCIPALES EN TIEMPO REAL
    # =========================================================================

    display(HTML("""
    <div class="card">
        <h3>INDICADORES CLAVE EN TIEMPO REAL</h3>
        <p>Estado actual del sistema y predicciones inmediatas</p>
    </div>
    """))

    # Obtener últimos datos
    ultimo_mes = df_clean.iloc[-1]
    capacidad_actual = ultimo_mes['capacidad_util_porcentaje']
    sedimentacion_actual = ultimo_mes['sedimentacion_porcentaje']
    dscr_actual = ultimo_mes['dscr']
    ebitda_actual = ultimo_mes['ebitda_millones_soles']

    # Predicción para el próximo mes
    datos_actuales = ultimo_mes[features].values.reshape(1, -1)
    datos_escalados = scaler.transform(datos_actuales)
    capacidad_predicha = model.predict(datos_escalados)[0]

    # Determinar alertas
    def determinar_alerta(capacidad):
        if capacidad < 58:
            return "🔴 ROJA", "#ff4444", "CRÍTICO"
        elif capacidad < 60:
            return "🟠 NARANJA", "#ff8800", "ALTO"
        elif capacidad < 62:
            return "🟡 AMARILLA", "#ffbb33", "MEDIO"
        else:
            return "🟢 VERDE", "#00C851", "BAJO"

    alerta_actual, color_actual, severidad_actual = determinar_alerta(capacidad_actual)
    alerta_predicha, color_predicha, severidad_predicha = determinar_alerta(capacidad_predicha)

    # Mostrar KPIs principales en grid
    kpi_html = f"""
    <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 20px; margin-bottom: 25px;">
        <div class="kpi-card" style="border-left: 6px solid {color_actual};">
            <h3>CAPACIDAD ACTUAL</h3>
            <h1 style="margin: 15px 0; font-size: 42px;">{capacidad_actual:.1f}%</h1>
            <div style="background: {color_actual}30; padding: 8px; border-radius: 20px;">
                <strong>{alerta_actual}</strong> - {severidad_actual}
            </div>
        </div>

        <div class="kpi-card" style="border-left: 6px solid {color_predicha};">
            <h3>PREDICCIÓN 1 MES</h3>
            <h1 style="margin: 15px 0; font-size: 42px;">{capacidad_predicha:.1f}%</h1>
            <div style="background: {color_predicha}30; padding: 8px; border-radius: 20px;">
                <strong>{alerta_predicha}</strong> - {severidad_predicha}
            </div>
        </div>

        <div class="kpi-card" style="border-left: 6px solid #ff9800;">
            <h3>SEDIMENTACIÓN</h3>
            <h1 style="margin: 15px 0; font-size: 42px;">{sedimentacion_actual:.1f}%</h1>
            <p>Tendencia: <strong>↗Creciente</strong></p>
        </div>

        <div class="kpi-card" style="border-left: 6px solid #2196f3;">
            <h3>DSCR ACTUAL</h3>
            <h1 style="margin: 15px 0; font-size: 42px;">{dscr_actual:.2f}</h1>
            <p>Mínimo: 1.10 | EBITDA: S/ {ebitda_actual:.1f}M</p>
        </div>
    </div>
    """

    display(HTML(kpi_html))

    # =========================================================================
    # SECCIÓN 2: GRÁFICOS INTERACTIVOS AVANZADOS
    # =========================================================================

    display(HTML("""
    <div class="card">
        <h3>ANÁLISIS VISUAL AVANZADO</h3>
        <p>Evolución histórica y tendencias operativas</p>
    </div>
    """))

    # Crear subplots con Plotly
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Evolución de Capacidad Útil vs Sedimentación',
            'Indicadores Financieros Clave',
            'Relación Capacidad vs Factores Operativos',
            'Tendencia de Riesgos Operativos'
        ),
        specs=[
            [{"secondary_y": True}, {}],
            [{"type": "scatter"}, {"type": "bar"}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.08
    )

    # Gráfico 1: Capacidad vs Sedimentación
    fig.add_trace(
        go.Scatter(x=df_clean['fecha'], y=df_clean['capacidad_util_porcentaje'],
                  name='Capacidad Útil', line=dict(color='#1f77b4', width=4)),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=df_clean['fecha'], y=df_clean['sedimentacion_porcentaje'],
                  name='Sedimentación', line=dict(color='#ff7f0e', width=4),
                  yaxis='y2'),
        row=1, col=1
    )

    # Líneas de alerta
    fig.add_hline(y=60, line_dash="dash", line_color="red", row=1, col=1,
                  annotation_text="Umbral Crítico 60%", annotation_position="bottom right")
    fig.add_hline(y=58, line_dash="dot", line_color="orange", row=1, col=1)

    # Gráfico 2: Indicadores financieros
    fig.add_trace(
        go.Scatter(x=df_clean['fecha'], y=df_clean['dscr'],
                  name='DSCR', line=dict(color='#2ca02c', width=3)),
        row=1, col=2
    )

    fig.add_trace(
        go.Scatter(x=df_clean['fecha'], y=df_clean['ebitda_millones_soles'],
                  name='EBITDA (S/)', line=dict(color='#d62728', width=3)),
        row=1, col=2
    )

    fig.add_hline(y=1.10, line_dash="dash", line_color="red", row=1, col=2,
                  annotation_text="Mínimo DSCR 1.10")

    # Gráfico 3: Relación capacidad vs morosidad (scatter)
    fig.add_trace(
        go.Scatter(x=df_clean['capacidad_util_porcentaje'],
                  y=df_clean['morosidad_porcentaje'],
                  mode='markers',
                  name='Capacidad vs Morosidad',
                  marker=dict(size=8, color=df_clean['sedimentacion_porcentaje'],
                            colorscale='Viridis', showscale=True,
                            colorbar=dict(title="Sedimentación"))),
        row=2, col=1
    )

    # Gráfico 4: Clientes en riesgo (bar)
    fig.add_trace(
        go.Bar(x=df_clean['fecha'], y=df_clean['clientes_riesgo_alto'],
               name='Clientes Riesgo Alto', marker_color='#ff6b6b'),
        row=2, col=2
    )

    # Actualizar layout
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="DASHBOARD EJECUTIVO - ANÁLISIS MULTIDIMENSIONAL",
        title_x=0.5,
        font=dict(size=12),
        plot_bgcolor='rgba(240,240,240,0.8)'
    )

    # Actualizar ejes
    fig.update_yaxes(title_text="Capacidad (%)", row=1, col=1)
    fig.update_yaxes(title_text="Sedimentación (%)", secondary_y=True, row=1, col=1)
    fig.update_yaxes(title_text="Valor", row=1, col=2)
    fig.update_yaxes(title_text="Morosidad (%)", row=2, col=1)
    fig.update_yaxes(title_text="Clientes Riesgo", row=2, col=2)
    fig.update_xaxes(title_text="Fecha", row=1, col=1)
    fig.update_xaxes(title_text="Fecha", row=1, col=2)
    fig.update_xaxes(title_text="Capacidad (%)", row=2, col=1)
    fig.update_xaxes(title_text="Fecha", row=2, col=2)

    fig.show()

    # =========================================================================
    # SECCIÓN 3: SIMULADOR DE ESCENARIOS ESTRATÉGICOS
    # =========================================================================

    display(HTML("""
    <div class="simulador-section">
        <div style="text-align: center; margin-bottom: 20px;">
            <h3>SIMULADOR DE ESCENARIOS ESTRATÉGICOS</h3>
            <p>Modifique los parámetros operativos para simular diferentes escenarios futuros</p>
        </div>
    </div>
    """))

    # Crear controles interactivos
    sedimentacion_slider = widgets.FloatSlider(
        value=42.0, min=35.0, max=50.0, step=0.5,
        description='Sedimentación (%):',
        style={'description_width': 'initial'},
        continuous_update=False
    )

    precipitacion_slider = widgets.FloatSlider(
        value=100.0, min=60.0, max=160.0, step=5.0,
        description='Precipitación (mm):',
        style={'description_width': 'initial'},
        continuous_update=False
    )

    ebitda_slider = widgets.FloatSlider(
        value=91.0, min=85.0, max=95.0, step=0.5,
        description='EBITDA (S/ millones):',
        style={'description_width': 'initial'},
        continuous_update=False
    )

    dscr_slider = widgets.FloatSlider(
        value=1.13, min=1.05, max=1.25, step=0.01,
        description='DSCR:',
        style={'description_width': 'initial'},
        continuous_update=False
    )

    morosidad_slider = widgets.FloatSlider(
        value=7.5, min=2.0, max=15.0, step=0.5,
        description='Morosidad (%):',
        style={'description_width': 'initial'},
        continuous_update=False
    )

    clientes_riesgo_slider = widgets.IntSlider(
        value=2, min=0, max=6, step=1,
        description='Clientes Riesgo Alto:',
        style={'description_width': 'initial'},
        continuous_update=False
    )

    # Crear pestañas CORREGIDAS
    tab_operativos = widgets.VBox([
        widgets.HTML("<div class='control-panel'><h4>PARÁMETROS OPERATIVOS</h4></div>"),
        sedimentacion_slider,
        precipitacion_slider
    ])

    tab_financieros = widgets.VBox([
        widgets.HTML("<div class='control-panel'><h4>PARÁMETROS FINANCIEROS</h4></div>"),
        ebitda_slider,
        dscr_slider
    ])

    tab_riesgo = widgets.VBox([
        widgets.HTML("<div class='control-panel'><h4>PARÁMETROS DE RIESGO</h4></div>"),
        morosidad_slider,
        clientes_riesgo_slider
    ])

    # Crear el tab widget CORREGIDO
    tab = widgets.Tab()
    tab.children = [tab_operativos, tab_financieros, tab_riesgo]
    tab.set_title(0, 'Operativos')
    tab.set_title(1, 'Financieros')
    tab.set_title(2, 'Riesgo')

    display(tab)

    # Botón de simulación
    simular_btn = widgets.Button(
        description='EJECUTAR SIMULACIÓN COMPLETA',
        button_style='success',
        style={'button_color': '#2a5298'},
        layout=widgets.Layout(width='300px', height='40px', margin='20px 0')
    )

    # Output para resultados
    resultado_output = widgets.Output()

    def ejecutar_simulacion_completa(_):
        with resultado_output:
            clear_output()

            # Crear datos de simulación
            datos_simulacion = {
                'sedimentacion_porcentaje': sedimentacion_slider.value,
                'precipitacion_mm': precipitacion_slider.value,
                'volumen_agua_entregado_millones_m3': 85.0,
                'dscr': dscr_slider.value,
                'ebitda_millones_soles': ebitda_slider.value,
                'morosidad_porcentaje': morosidad_slider.value,
                'clientes_riesgo_alto': clientes_riesgo_slider.value,
                'mes': 6
            }

            # Convertir a DataFrame y predecir
            sim_df = pd.DataFrame([datos_simulacion])[features]
            sim_escalado = scaler.transform(sim_df)
            prediccion = model.predict(sim_escalado)[0]

            # Análisis completo del escenario
            alerta, color, severidad = determinar_alerta(prediccion)

            # Generar recomendaciones específicas
            if prediccion < 58:
                recomendaciones = [
                    "Convocar junta directiva de emergencia",
                    "Activar protocolos de contingencia operativa",
                    "Revisar cumplimiento de covenants financieros",
                    "Comunicar situación a entidades reguladoras"
                ]
                impacto = "CRÍTICO - Acción inmediata requerida"
            elif prediccion < 60:
                recomendaciones = [
                    "Revisar programa de mantenimiento urgente",
                    "Evaluar opciones de financiamiento adicional",
                    "Optimizar operaciones inmediatamente",
                    "Monitoreo intensivo 24/7"
                ]
                impacto = "ALTO - Atención prioritaria requerida"
            elif prediccion < 62:
                recomendaciones = [
                    "Implementar mantenimiento preventivo",
                    "Revisar eficiencia operativa",
                    "Fortalecer gestión de cartera",
                    "Monitoreo semanal intensivo"
                ]
                impacto = "MEDIO - Acciones preventivas recomendadas"
            else:
                recomendaciones = [
                    "Continuar monitoreo rutinario",
                    "Mantener programas de mantenimiento",
                    "Optimizar procesos operativos",
                    "Revisión trimestral de indicadores"
                ]
                impacto = "BAJO - Situación estable"

            # Mostrar resultados de simulación
            display(HTML(f"""
            <div class="card" style="border-left: 6px solid {color};">
                <div style="text-align: center; padding: 20px;">
                    <h1 style="color: {color}; font-size: 52px; margin: 10px 0;">{prediccion:.1f}%</h1>
                    <h2 style="color: {color}; margin: 10px 0;">NIVEL DE ALERTA: {alerta}</h2>
                    <h3 style="color: #666;">{impacto}</h3>
                </div>

                <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin: 20px 0;">
                    <div>
                        <h4>PARÁMETROS SIMULADOS:</h4>
                        <ul style="list-style: none; padding: 0;">
                            <li>Sedimentación: <strong>{sedimentacion_slider.value}%</strong></li>
                            <li>Precipitación: <strong>{precipitacion_slider.value} mm</strong></li>
                            <li>EBITDA: <strong>S/ {ebitda_slider.value}M</strong></li>
                            <li>DSCR: <strong>{dscr_slider.value}</strong></li>
                            <li>Morosidad: <strong>{morosidad_slider.value}%</strong></li>
                            <li>Clientes Riesgo: <strong>{clientes_riesgo_slider.value}</strong></li>
                        </ul>
                    </div>

                    <div>
                        <h4>RECOMENDACIONES ESTRATÉGICAS:</h4>
                        <ul style="list-style: none; padding: 0;">
                            {''.join([f'<li>{rec}</li>' for rec in recomendaciones])}
                        </ul>
                    </div>
                </div>

                <div style="background-color: {color}20; padding: 20px; border-radius: 8px; margin-top: 15px;">
                    <h4>IMPACTO ESPERADO:</h4>
                    <p>La implementación de estas recomendaciones podría mejorar la capacidad en <strong>3-5%</strong>
                    en los próximos 3 meses y reducir el riesgo operativo en <strong>15-25%</strong>.</p>
                </div>
            </div>
            """))

    # Conectar botón y mostrar controles
    simular_btn.on_click(ejecutar_simulacion_completa)

    display(widgets.VBox([
        widgets.HBox([simular_btn], layout=widgets.Layout(justify_content='center')),
        resultado_output
    ]))

    # Ejecutar simulación inicial
    ejecutar_simulacion_completa(None)

    # =========================================================================
    # SECCIÓN 4: PREDICCIONES POR LOTES Y PROYECCIONES
    # =========================================================================

    display(HTML("""
    <div class="card">
        <h3>PROYECCIONES ESTRATÉGICAS - PRÓXIMOS 6 MESES</h3>
        <p>Análisis predictivo automatizado basado en tendencias actuales</p>
    </div>
    """))

    # Simular datos futuros
    ultimos_datos = df_clean[features].iloc[-6:].copy()

    # Factores de tendencia
    factores_tendencia = {
        'sedimentacion_porcentaje': 1.015,
        'precipitacion_mm': 0.98,
        'volumen_agua_entregado_millones_m3': 0.995,
        'dscr': 0.998,
        'ebitda_millones_soles': 0.997,
        'morosidad_porcentaje': 1.02
    }

    datos_futuros = []
    fechas_futuras = []
    fecha_actual = df_clean['fecha'].iloc[-1]

    for i in range(6):
        nueva_fecha = fecha_actual + pd.DateOffset(months=i+1)
        fechas_futuras.append(nueva_fecha)

        if i == 0:
            datos_mes = ultimos_datos.iloc[i].copy()
        else:
            datos_mes = datos_futuros[-1].copy()

        for variable, factor in factores_tendencia.items():
            if variable in datos_mes.index:
                datos_mes[variable] *= factor

        datos_mes['mes'] = nueva_fecha.month
        datos_futuros.append(datos_mes)

    # Hacer predicciones
    futuro_df = pd.DataFrame(datos_futuros)
    futuro_escalado = scaler.transform(futuro_df[features])
    predicciones = model.predict(futuro_escalado)

    # Crear tabla de resultados
    resultados = []
    for i, (fecha, pred) in enumerate(zip(fechas_futuras, predicciones)):
        alerta, color, severidad = determinar_alerta(pred)

        resultados.append({
            'Mes': fecha.strftime('%b-%Y'),
            'Sedimentación': f"{datos_futuros[i]['sedimentacion_porcentaje']:.1f}%",
            'Precipitación': f"{datos_futuros[i]['precipitacion_mm']:.1f} mm",
            'DSCR': f"{datos_futuros[i]['dscr']:.2f}",
            'Capacidad_Predicha': f"{pred:.1f}%",
            'Alerta': alerta,
            'Severidad': severidad
        })

    # Mostrar tabla con estilo
    tabla_html = """
    <div style="overflow-x: auto;">
        <table border="1" style="border-collapse: collapse; width: 100%; text-align: center; font-size: 14px;">
            <thead>
                <tr style="background: linear-gradient(90deg, #1e3c72, #2a5298); color: white;">
                    <th style="padding: 12px;">Mes</th>
                    <th style="padding: 12px;">Sedimentación</th>
                    <th style="padding: 12px;">Precipitación</th>
                    <th style="padding: 12px;">DSCR</th>
                    <th style="padding: 12px;">Capacidad Predicha</th>
                    <th style="padding: 12px;">Nivel de Alerta</th>
                </tr>
            </thead>
            <tbody>
    """

    for fila in resultados:
        color_fondo = ""
        if 'ROJA' in fila['Alerta']:
            color_fondo = "background-color: #ffebee;"
        elif 'NARANJA' in fila['Alerta']:
            color_fondo = "background-color: #fff3e0;"
        elif 'AMARILLA' in fila['Alerta']:
            color_fondo = "background-color: #fffde7;"
        else:
            color_fondo = "background-color: #f1f8e9;"

        tabla_html += f"""
            <tr style="{color_fondo}">
                <td style="padding: 10px;"><strong>{fila['Mes']}</strong></td>
                <td style="padding: 10px;">{fila['Sedimentación']}</td>
                <td style="padding: 10px;">{fila['Precipitación']}</td>
                <td style="padding: 10px;">{fila['DSCR']}</td>
                <td style="padding: 10px;"><strong>{fila['Capacidad_Predicha']}</strong></td>
                <td style="padding: 10px;"><strong>{fila['Alerta']}</strong></td>
            </tr>
        """

    tabla_html += "</tbody></table></div>"
    display(HTML(tabla_html))

    # Resumen ejecutivo de proyecciones
    alertas_rojas = sum(1 for p in predicciones if p < 58)
    alertas_naranjas = sum(1 for p in predicciones if 58 <= p < 60)

    display(HTML(f"""
    <div class="card">
        <h3>RESUMEN EJECUTIVO DE PROYECCIONES</h3>
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
            <div>
                <h4>DISTRIBUCIÓN DE ALERTAS:</h4>
                <ul style="list-style: none; padding: 0;">
                    <li>🔴 Alertas CRÍTICAS: <strong>{alertas_rojas} meses</strong></li>
                    <li>🟠 Alertas ALTAS: <strong>{alertas_naranjas} meses</strong></li>
                    <li>🟡 Alertas MEDIAS: <strong>{sum(1 for p in predicciones if 60 <= p < 62)} meses</strong></li>
                    <li>🟢 Alertas BAJAS: <strong>{sum(1 for p in predicciones if p >= 62)} meses</strong></li>
                </ul>
            </div>
            <div>
                <h4>RECOMENDACIÓN GLOBAL:</h4>
                <p>{'Se requiere intervención estratégica inmediata' if alertas_rojas > 2 else 'Plan de acción preventivo recomendado' if alertas_rojas > 0 else 'Monitoreo continuo suficiente'}</p>
                <p><strong>Capacidad promedio proyectada: {np.mean(predicciones):.1f}%</strong></p>
            </div>
        </div>
    </div>
    """))

# Ejecutar dashboard interactivo completo
crear_dashboard_interactivo_completo(df_clean, model, scaler, features)

Mes,Sedimentación,Precipitación,DSCR,Capacidad Predicha,Nivel de Alerta
Oct-2025,45.7%,124.0 mm,1.05,49.8%,🔴 ROJA
Nov-2025,46.4%,121.6 mm,1.05,49.8%,🔴 ROJA
Dec-2025,47.1%,119.1 mm,1.04,49.8%,🔴 ROJA
Jan-2026,47.8%,116.7 mm,1.04,49.8%,🔴 ROJA
Feb-2026,48.5%,114.4 mm,1.04,49.8%,🔴 ROJA
Mar-2026,49.2%,112.1 mm,1.04,49.8%,🔴 ROJA
